# CUDA INSTALL

In [ ]:
!nvidia-smi

Thu Nov 24 14:46:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-kx4oekj7
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-kx4oekj7


In [ ]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [ ]:
import cv2
cv2.__version__

'4.6.0'

In [ ]:
!find .. -name 'libopencv*'

../var/lib/dpkg/info/libopencv-ts-dev:amd64.list
../var/lib/dpkg/info/libopencv-highgui-dev:amd64.md5sums
../var/lib/dpkg/info/libopencv-imgproc-dev:amd64.md5sums
../var/lib/dpkg/info/libopencv-flann3.2:amd64.md5sums
../var/lib/dpkg/info/libopencv-flann-dev:amd64.md5sums
../var/lib/dpkg/info/libopencv-superres3.2:amd64.md5sums
../var/lib/dpkg/info/libopencv-photo3.2:amd64.md5sums
../var/lib/dpkg/info/libopencv-contrib3.2:amd64.shlibs
../var/lib/dpkg/info/libopencv-photo-dev:amd64.list
../var/lib/dpkg/info/libopencv-viz3.2:amd64.md5sums
../var/lib/dpkg/info/libopencv-features2d3.2:amd64.md5sums
../var/lib/dpkg/info/libopencv-objdetect3.2:amd64.list
../var/lib/dpkg/info/libopencv-stitching3.2:amd64.list
../var/lib/dpkg/info/libopencv-imgcodecs3.2:amd64.md5sums
../var/lib/dpkg/info/libopencv-calib3d-dev:amd64.list
../var/lib/dpkg/info/libopencv3.2-jni.list
../var/lib/dpkg/info/libopencv-video3.2:amd64.md5sums
../var/lib/dpkg/info/libopencv-shape3.2:amd64.shlibs
../var/lib/dpkg/info/libope

In [ ]:
!pkg-config --cflags --libs opencv

-I/usr/include/opencv -lopencv_shape -lopencv_stitching -lopencv_superres -lopencv_videostab -lopencv_aruco -lopencv_bgsegm -lopencv_bioinspired -lopencv_ccalib -lopencv_datasets -lopencv_dpm -lopencv_face -lopencv_freetype -lopencv_fuzzy -lopencv_hdf -lopencv_line_descriptor -lopencv_optflow -lopencv_video -lopencv_plot -lopencv_reg -lopencv_saliency -lopencv_stereo -lopencv_structured_light -lopencv_phase_unwrapping -lopencv_rgbd -lopencv_viz -lopencv_surface_matching -lopencv_text -lopencv_ximgproc -lopencv_calib3d -lopencv_features2d -lopencv_flann -lopencv_xobjdetect -lopencv_objdetect -lopencv_ml -lopencv_xphoto -lopencv_highgui -lopencv_videoio -lopencv_imgcodecs -lopencv_photo -lopencv_imgproc -lopencv_core


In [ ]:
%%cu
#include <stdio.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
__global__
void cuda_kernel()
{
    printf("Hello World!");
}
int main()
{
    cuda_kernel <<< 1, 1 >>> ();
    cudaDeviceSynchronize();    
    cudaDeviceReset();  
    return 0;
}

Hello World!


In [ ]:
%%cu
#include <cstdio>
#include <iostream>
  
using namespace std;
  
__global__ void maxi(int* a, int* b, int n)
{
    int block = 256 * blockIdx.x;
    int max = 0;
  
    for (int i = block; i < min(256 + block, n); i++) {
  
        if (max < a[i]) {
            max = a[i];
        }
    }
    b[blockIdx.x] = max;
}
  
int main()
{
  
    int n;
    n = 3 >> 2;
    int a[n];
  
    for (int i = 0; i < n; i++) {
        a[i] = rand() % n;
        cout << a[i] << "\t";
    }
  
    cudaEvent_t start, end;
    int *ad, *bd;
    int size = n * sizeof(int);
    cudaMalloc(&ad, size);
    cudaMemcpy(ad, a, size, cudaMemcpyHostToDevice);
    int grids = ceil(n * 1.0f / 256.0f);
    cudaMalloc(&bd, grids * sizeof(int));
  
    dim3 grid(grids, 1);
    dim3 block(1, 1);
  
    cudaEventCreate(&start);
    cudaEventCreate(&end);
    cudaEventRecord(start);
  
    while (n > 1) {
        maxi<<<grids, block>>>(ad, bd, n);
        n = ceil(n * 1.0f / 256.0f);
        cudaMemcpy(ad, bd, n * sizeof(int), cudaMemcpyDeviceToDevice);
    }
  
    cudaEventRecord(end);
    cudaEventSynchronize(end);
  
    float time = 0;
    cudaEventElapsedTime(&time, start, end);
  
    int ans[2];
    cudaMemcpy(ans, ad, 4, cudaMemcpyDeviceToHost);
  
    cout << "The maximum element is : " << ans[0] << endl;
  
    cout << "The time required : ";
    cout << time << endl;
}

The maximum element is : 476473360
The time required : 0.0024



In [ ]:
!sudo apt install -y g++ cmake make git libgtk2.0-dev pkg-config

Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.1-9.1ubuntu1).
make set to manually installed.
pkg-config is already the newest version (0.29.1-0ubuntu2).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
g++ is already the newest version (4:7.4.0-1ubuntu2.3).
g++ set to manually installed.
git is already the newest version (1:2.17.1-1ubuntu0.13).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  autoconf automake autopoint autotools-dev debhelper dh-autoreconf
  dh-strip-nondeterminism file gettext gettext-base gir1.2-atk-1.0
  gir1.2-freedesktop gir1.2-gdkpixbuf-2.0 gir1.2-gtk-2.0 gir1.2-pango-1.0
  intltool-debian libarchive-cpio-perl libarchive-zip-perl libatk1.0-dev
  libcairo-script-interpreter2 libcairo2-dev libfile-stripnondeterminism-per

In [ ]:
!git clone https://github.com/opencv/opencv.git

Cloning into 'opencv'...
remote: Enumerating objects: 311532, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 311532 (delta 33), reused 45 (delta 10), pack-reused 311434
Receiving objects: 100% (311532/311532), 499.11 MiB | 16.30 MiB/s, done.
Resolving deltas: 100% (216853/216853), done.
Checking out files: 100% (7264/7264), done.


In [ ]:
%cd src/

/content/src


In [ ]:
!mkdir -p build && cd build

In [ ]:
%cd /content/src/build/

/content/src/build


In [ ]:
!cmake /content/src/opencv

-- 'Release' build type is used by default. Use CMAKE_BUILD_TYPE to specify build type (Release or Debug)
-- The CXX compiler identification is GNU 7.5.0
-- The C compiler identification is GNU 7.5.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detected processor: x86_64
-- Found PythonInterp: /usr/bin/python2.7 (found suitable version "2.7.17", minimum required is "2.7") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython2.7.so (found suitable exact version "2.7.17") 
-- Found PythonInterp: /usr/bin/python3 (found suitable version "3.7.15", minimum required is "3.2") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libp

In [ ]:
!make -j4

[  0%] Built target opencv_dnn_plugins
[  0%] Built target opencv_videoio_plugins
[  0%] Built target opencv_highgui_plugins
[  0%] Building C object 3rdparty/ippiw/CMakeFiles/ippiw.dir/src/iw_core.c.o
[  0%] Building C object 3rdparty/ittnotify/CMakeFiles/ittnotify.dir/src/ittnotify/ittnotify_static.c.o
[  0%] Building C object 3rdparty/quirc/CMakeFiles/quirc.dir/src/decode.c.o
[  0%] Building CXX object 3rdparty/protobuf/CMakeFiles/libprotobuf.dir/src/google/protobuf/any_lite.cc.o
[  0%] Building C object 3rdparty/ippiw/CMakeFiles/ippiw.dir/src/iw_image.c.o
[  0%] Building C object 3rdparty/ippiw/CMakeFiles/ippiw.dir/src/iw_image_color_convert_all.c.o
[  1%] Building C object 3rdparty/ippiw/CMakeFiles/ippiw.dir/src/iw_image_color_convert_rgbs.c.o
[  1%] Building C object 3rdparty/ittnotify/CMakeFiles/ittnotify.dir/src/ittnotify/jitprofiling.c.o
[  1%] Building CXX object 3rdparty/protobuf/CMakeFiles/libprotobuf.dir/src/google/protobuf/arena.cc.o
[  2%] Linking C static library ../lib

In [ ]:
%cd /content/src/build
!sudo make install

/content/src/build
[  0%] Built target opencv_dnn_plugins
[  0%] Built target opencv_highgui_plugins
[  0%] Built target opencv_videoio_plugins
Consolidate compiler generated dependencies of target ippiw
[  2%] Built target ippiw
Consolidate compiler generated dependencies of target ittnotify
[  3%] Built target ittnotify
Consolidate compiler generated dependencies of target opencv_core
[ 11%] Built target opencv_core
Consolidate compiler generated dependencies of target opencv_imgproc
[ 18%] Built target opencv_imgproc
Consolidate compiler generated dependencies of target opencv_imgcodecs
[ 20%] Built target opencv_imgcodecs
Consolidate compiler generated dependencies of target opencv_videoio
[ 21%] Built target opencv_videoio
Consolidate compiler generated dependencies of target opencv_highgui
[ 21%] Built target opencv_highgui
Consolidate compiler generated dependencies of target opencv_ts
[ 22%] Built target opencv_ts
Consolidate compiler generated dependencies of target opencv_tes

# Blurfaces

In [ ]:
%cd ..

/content/src


In [ ]:
!mkdir -p 0Prueba

In [ ]:
%cd /content/src/0Prueba/

/content/src/0Prueba


In [ ]:
%%writefile blurFaces.cu
#include <opencv2/opencv.hpp>
#include <opencv2/core.hpp>
#include <opencv2/videoio.hpp>
#include <opencv2/highgui.hpp>
#include <iostream>
#include <stdio.h>
#include <bits/stdc++.h>
#include <time.h>
#include <cuda_runtime.h>

using namespace cv;
using namespace std;

#define SHARED_SIZE 128

__global__ void gaussianblur(uchar *frame, int x, int y, int h, int w, int kernelSize, int rows, int cols){
    
    __shared__ uchar temp[SHARED_SIZE*SHARED_SIZE*3];
    int initial_row = y-kernelSize/2;
    for(int col = x-kernelSize/2;col<=x+w+kernelSize/2;col++){
        temp[threadIdx.x*SHARED_SIZE*3+(col-(x-kernelSize/2))*3] = frame[(initial_row+threadIdx.x)*3*cols+col*3];
        temp[threadIdx.x*SHARED_SIZE*3+(col-(x-kernelSize/2))*3+1] = frame[(initial_row+threadIdx.x)*3*cols+col*3+1];
        temp[threadIdx.x*SHARED_SIZE*3+(col-(x-kernelSize/2))*3+2] = frame[(initial_row+threadIdx.x)*3*cols+col*3+2];
    }
    // Recorrido por filas hasta el total de la altura
    __syncthreads();
    R = 0, G = 0, B = 0;
    for(int row = 0;row<=h+kernelSize-1;row++){
        if(row>=kernelSize){
            temp[(row-kernelSize/2)*SHARED_SIZE*3 + threadIdx.x*3] = R;
            temp[(row-kernelSize/2)*SHARED_SIZE*3 + threadIdx.x*3+1] = G;
            temp[(row-kernelSize/2)*SHARED_SIZE*3 + threadIdx.x*3+2] = B;
            R -= temp[(row-kernelSize)*SHARED_SIZE*3 + threadIdx.x*3]/kernelSize;
            G -= temp[(row-kernelSize)*SHARED_SIZE*3 + threadIdx.x*3+1]/kernelSize;
            B -= temp[(row-kernelSize)*SHARED_SIZE*3 + threadIdx.x*3+2]/kernelSize;
        }
        R += temp[row*SHARED_SIZE*3+threadIdx.x*3]/kernelSize;
        G += temp[row*SHARED_SIZE*3+threadIdx.x*3+1]/kernelSize;
        B += temp[row*SHARED_SIZE*3+threadIdx.x*3+2]/kernelSize;
    }
    // Recorrido por columnas hasta el total del ancho
    __syncthreads();
    uchar R = 0, G = 0, B = 0;
    for(int col = 0;col<=w+kernelSize-1;col++){
        if(col>=kernelSize){
            temp[threadIdx.x*SHARED_SIZE*3 + (col-kernelSize/2)*3] = R;
            temp[threadIdx.x*SHARED_SIZE*3 + (col-kernelSize/2)*3+1] = G;
            temp[threadIdx.x*SHARED_SIZE*3 + (col-kernelSize/2)*3+2] = B;
            R -= temp[threadIdx.x*SHARED_SIZE*3 + (col-kernelSize)*3]/kernelSize;
            G -= temp[threadIdx.x*SHARED_SIZE*3 + (col-kernelSize)*3+1]/kernelSize;
            B -= temp[threadIdx.x*SHARED_SIZE*3 + (col-kernelSize)*3+2]/kernelSize;
        }
        R += temp[threadIdx.x*SHARED_SIZE*3+col*3]/kernelSize;
        G += temp[threadIdx.x*SHARED_SIZE*3+col*3+1]/kernelSize;
        B += temp[threadIdx.x*SHARED_SIZE*3+col*3+2]/kernelSize;
    }
    
    __syncthreads();

    for(int col = x-kernelSize/2;col<=x+w+kernelSize/2;col++){
        frame[(initial_row+threadIdx.x)*3*cols+col*3] = temp[threadIdx.x*SHARED_SIZE*3+(col-(x-kernelSize/2))*3] ;
        frame[(initial_row+threadIdx.x)*3*cols+col*3+1] = temp[threadIdx.x*SHARED_SIZE*3+(col-(x-kernelSize/2))*3+1] ;
        frame[(initial_row+threadIdx.x)*3*cols+col*3+2] = temp[threadIdx.x*SHARED_SIZE*3+(col-(x-kernelSize/2))*3+2] ;
    }
}


int main (int argc, char *argv[]) {
    char path[100] = "/content/src/0Prueba/";
    char path2[100] = "/content/src/0Prueba/";
    strcat(path,argv[1]);
    strcat(path2,argv[2]);

    VideoCapture cap(path);
    int frame_w = (int)(cap.get(3));
    int frame_h = (int)(cap.get(4));
    Size frame_size(frame_w, frame_h);
    int fps = 20;
    int totalFrames = (int)cap.get(7);

    VideoWriter output(path2, VideoWriter::fourcc('M', 'P', '4', 'V'),fps, frame_size);

    cout<<"Total frames "<<totalFrames<<endl;
    cout<<"Frame w "<<frame_w<<endl;
    cout<<"Frame h "<<frame_h<<endl;

    if(!cap.isOpened()){
        cout << "Error opening video stream or file" << endl;
        return -1;
    }

    int kernelSize = 15;
    double div = (double)1/(kernelSize*kernelSize);
    
    // Variable para almacenar coordenadas y medidas de rostros
    vector<Rect> faces;
    CascadeClassifier face_cascade;
    face_cascade.load("/content/src/0Prueba/haarcascade_frontalface_alt.xml");
    cudaError_t err = cudaSuccess;
    clock_t start = clock();
    // Se itera por todos los frames del video
    int cont = 0;
    int blocks_num = 16;

    while(cap.isOpened()){
        // Establece fotograma a analizar
        Mat frame;
        bool isSuccess = cap.read(frame);
        
        // Verifica si el frame se leyó con éxito
        if (!isSuccess){
            cout << "Stream disconnected" << endl;
            break;
        }
            
        if (frame.empty())break;
        face_cascade.detectMultiScale(frame, faces, 1.1, 3,0);
        if(faces.empty()){
          output.write(frame);
          continue;
        }

        uchar *h_frame, *d_frame;
        int size,threads_num = 0;
        h_frame = frame.isContinuous()? frame.data: frame.clone().data;
        uint length = frame.total()*frame.channels();        
        size = sizeof(uchar)*length;
        err = cudaMalloc((void **)&d_frame, size);
        
        if (err != cudaSuccess){
            fprintf(stderr, "Fallo en vector (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }
        err = cudaMemcpy(d_frame, h_frame, size, cudaMemcpyHostToDevice);
        if (err != cudaSuccess){
            fprintf(stderr, "Fallo al copiar el vector de altura para (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }
        for(Rect r:faces){
            if(r.h+kernelSize>=SHARED_SIZE || r.w+kernelSize>=SHARED_SIZE)continue;
            threads_num = r.h+kernelSize-1;
            blur<<<blocks_num,threads_num>>>(d_frame,r.x,r.y,r.h,r.w,kernelSize,frame_h,frame_w);
        }
        cudaDeviceSynchronize();
       
        err = cudaGetLastError();
        if (err != cudaSuccess){
            fprintf(stderr, "Falo al cargar el kernel (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        err = cudaMemcpy(h_frame, d_frame, size, cudaMemcpyDeviceToHost);
        if (err != cudaSuccess){
            fprintf(stderr, "Fallo al copiar el vector C en (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        err = cudaFree(d_frame);
        if (err != cudaSuccess){
            fprintf(stderr, "Fallo al liberar el vectro C en (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        faces.clear();
        output.write(frame);

        //cout<<(double)cont*100/totalFrames<<"%"<<endl;
        cont++;      
    }
    
    err = cudaDeviceReset();
    if (err != cudaSuccess){
        fprintf(stderr, "Fallo al desinicializar error=%s\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }
    // Finaliza el programa
    start = clock() - start;
    
    cap.release();
    destroyAllWindows();
    cout << "TIempo total: " << float(start)/CLOCKS_PER_SEC << endl;
    return 0;
}

Overwriting blurFaces.cu


In [ ]:
!cmake .

CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- Configuring done
CMake Warning (dev) in CMakeLists.txt:
  Policy CMP0104 is not set: CMAKE_CUDA_ARCHITECTURES now detected for NVCC,
  empty CUDA_ARCHITECTURES not allowed.  Run "cmake --help-policy CMP0104"
  for policy details.  Use the cmake_policy command to set the policy and
  suppress this warning.

  CUDA_ARCHITECTURES is empty for target "blurFaces".
This warning is for project developers.  Use -Wno-dev to suppress it.

CMake Warning (dev) in CMakeLists.txt:
  Policy CMP0104 is not set: CMAKE_CUDA_ARCHITECTURES now detected for NVCC,
  empty CUDA_ARCHITECTURES not allowed.  Run "cmake --help-policy CMP0104"
  for policy details.  Use the cmake_policy command to set the 

In [ ]:
!make

Consolidate compiler generated dependencies of target blurFaces
[ 50%] Building CUDA object CMakeFiles/blurFaces.dir/blurFaces.cu.o
[100%] Linking CUDA executable blurFaces
[100%] Built target blurFaces


In [ ]:
!time ./blurFaces video_peq.mp4 NumBloq16_video_salida.mp4

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 13 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
Total frames 272
Frame width 960
Frame height 540
Stream disconnected
TIempo total: 187.535

real	1m39.472s
user	3m6.453s
sys	0m1.214s
